In [1]:
#Import Modules
import folium
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement
import pandas as pd
import time
import requests

In [3]:
#Read in the CSV
cities_df = pd.read_csv("/Users/aidanobrien/Documents/Jupyter Notebooks/Canadian Cities Notebook/canadacities.csv")

#Fill empty cells in the lived and visited column with "no"
cities_df['lived'] = cities_df['lived'].fillna("no")
cities_df['visited'] = cities_df['visited'].fillna("no")

# A function to return different colour strings based on the values of the "Lived" and "Visited" Columns. If "Lived" == Yes then pink, etc.
def select_marker_colour(row):
    if row['lived'] == 'yes':
        return 'pink'
    elif row['lived'] == "no" and row['visited'] == 'yes':
        return 'orange'
    return 'lightblue'

#Add a new column to the data with the Colours that were assigned based on the Lived and Visited columns
cities_df['colour'] = cities_df.apply(select_marker_colour, axis=1)

#create a list of lat,longs 
locations = cities_df[['lat', 'lng']]
locationlist = locations.values.tolist()

#Check that the dataframe is looking good
cities_df.head(3)

,city,city_ascii,province_id,province_name,lat,lng,population,density,timezone,ranking,postal,id,lived,visited,colour
0,Toronto,Toronto,ON,Ontario,43.7417,-79.3733,5429524,4334.4,America/Toronto,1,M5T M5V M5P M5S M5R M5E M5G M5A M5C M5B M5M M5...,1124279679,yes,yes,pink
1,Montréal,Montreal,QC,Quebec,45.5089,-73.5617,3519595,3889.0,America/Montreal,1,H1X H1Y H1Z H1P H1R H1S H1T H1V H1W H1H H1J H1...,1124586170,no,yes,orange
2,Vancouver,Vancouver,BC,British Columbia,49.2500,-123.1000,2264823,5492.6,America/Vancouver,1,V6Z V6S V6R V6P V6N V6M V6L V6K V6J V6H V6G V6...,1124825478,no,no,lightblue


In [ ]:
#Create a map with folium
cluster_map = folium.Map(
    location = [43.6532, -79.3832],
    tiles = 'OpenStreetMap',
    zoom_start = 8,
    width = 500,
    height = 300,
    control_scale = True)

#Create clustered markers where we can add the city markers
marker_cluster = MarkerCluster().add_to(cluster_map)

#Create markers from each cities lat/long. The popup and tool tip also include the population data.
for _, row in cities_df.iloc[1:].iterrows():
    folium.Marker(
        location = [row['lat'], row['lng']],
        popup = ["City: " + row['city'],"Population: " + str(row['population'])],
        tooltip = ["City: " + row['city'],"Population: " + str(row['population'])],
        icon = folium.Icon(color = row['colour'])
    ).add_to(marker_cluster)

#Add some different tiles
folium.TileLayer('Stamen Toner').add_to(cluster_map)
folium.TileLayer('Cartodbdark_matter').add_to(cluster_map)
folium.TileLayer('Stamen Terrain').add_to(cluster_map)

folium.LayerControl().add_to(cluster_map)

In [ ]:
#draw the map
cluster_map

In [ ]:
#Lets make a second map without the clusters to get a better visual
my_map = folium.Map(
    location = [43.6532, -79.3832],
    tiles = 'OpenStreetMap',
    zoom_start = 8,
    width = 750,
    height = 500,
    control_scale = True)

#iterate through cities lat/lngs agains
for _, row in cities_df.iloc[1:].iterrows():
    folium.Marker(
        location = [row['lat'], row['lng']],
        popup = ["City: " + row['city'],"Population: " + str(row['population'])],
        tooltip = ["City: " + row['city'],"Population: " + str(row['population'])],
        icon = folium.Icon(color = row['colour'])
    ).add_to(my_map)


#Tile Selector
folium.TileLayer('Stamen Toner').add_to(my_map)
folium.TileLayer('Cartodbdark_matter').add_to(my_map)
folium.TileLayer('Stamen Terrain').add_to(my_map)

folium.LayerControl().add_to(my_map)


#Lets use some html to make a draggable legend on our map
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:pink;opacity:0.7;'></span>Lived</li>
    <li><span style='background:orange;opacity:0.7;'></span>Visited</li>
    <li><span style='background:cyan;opacity:0.7;'></span>Yet to Visit</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""


#Add the legend to the map
macro = MacroElement()
macro._template = Template(template)

my_map.get_root().add_child(macro)

#draw map
my_map